# Lecture 3: Basic Models in TensorFlow
## 1. Overview
1. Review
1. Linear regression in TensorFlow
1. Optimizers
1. Logistic regression on MNIST
1. Loss functions


[github](https://github.com/chiphuyen/tf-stanford-tutorials)
- examples: 
    - 03_linear_regression_starter.py
    - 03_logistic_regression_mnist_starter.py
- data: fire_theft.xls

## 2. Linear regression
### example: Chicago fire theft
- X: number of incidents of fire
- Y: number of incidents of theft
- goal: Predict Y from X
- model: $Y_{predicted} = w * X + b$  
$(Y - Y_{predicted})^2$

### Phase 1: Assemble graph
- Step 1: Read in data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import xlrd

In [ ]:
DATA_FILE = 'data/fire_theft.xls'

book = xlrd.open_workbook(DATA_FILE, encoding_override='utf-8')
sheet = book.sheet_by_index(0)
data = np.asarray([sheet.row_values(i) for i in range(1, sheet.nrows)])
n_samples = sheet.nrows - 1

- Step 2: Create placeholders for inputs and labels

In [ ]:
X = tf.placeholder(tf.float32, name='X')
Y = tf.placeholder(tf.float32, name='Y')

- Step 3: Create weight and bias  
`tf.Variable(initial_value=None, trainable=True, collections=None, name=None, dtype=None, ...)`

In [ ]:
w = tf.Variable(0.0, name='weights')
b = tf.Variable(0.0, name='bias')

- Step 4: Build model to predict Y

In [ ]:
Y_predicted = X * w + b 

- Step 5: Specify loss function

In [ ]:
loss = tf.square(Y - Y_predicted, name='loss')

- Step 6: Create optimizer

In [ ]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(loss)

### Phase 2: Train the model
- Initialize variables
- Run optimizer op
（with data fed into placeholders for inputs and labels）  

### See model in TensorBoard
- Step 1: `writer = tf.summary.FileWriter('./my_graph/03/linear_reg', sess.graph)`
- Step 2: `$ tensorboard --logdir='./my_graph'`

In [ ]:
with tf.Session() as sess:
	# Step 7: initialize the necessary variables, in this case, w and b
	sess.run(tf.global_variables_initializer()) 
	
	writer = tf.summary.FileWriter('./my_graph/03/linear_reg', sess.graph)
	
	# Step 8: train the model
	for i in range(100): # train the model 100 times
		total_loss = 0
		for x, y in data:
			# Session runs train_op and fetch values of loss
			_, l = sess.run([optimizer, loss], feed_dict={X: x, Y:y}) 
			total_loss += l
		print('Epoch {0}: {1}'.format(i, total_loss/n_samples))

	# close the writer when you're done using it
	writer.close() 
	
	# Step 9: output the values of w and b
	w_value, b_value = sess.run([w, b]) 

### Plot the results with matplotlib


In [ ]:
X, Y = data.T[0], data.T[1]
plt.plot(X, Y, 'bo', label='Real data')
plt.plot(X, X * w_value + b_value, 'r', label='Predicted data')
plt.legend()
plt.show()

## 3. Optimizer
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(loss)
    _, l = sess.run([optimizer, loss], feed_dict={X: x, Y:y})
Session looks at all **trainable** variables that loss depends on and update them
### list of optimizers in TF
- tf.train.GradientDescentOptimizer
- tf.train.AdagradOptimizer
- tf.train.MomentumOptimizer
- tf.train.AdamOptimizer
- tf.train.ProximalGradientDescentOptimizer
- tf.train.ProximalAdagradOptimizer
- tf.train.RMSPropOptimizer  
And more  
### improve model: Huber loss
- Robust to outliers
- Intuition:  
    - if the difference between the predicted value and the real value is small, square it  
    - If it’s large, take its absolute value
- Implementation: can't write $if\ Y - Y_{predicted} < delta$

In [ ]:
def huber_loss(labels, predictions, delta=1.0):
    residual = tf.abs(predictions - labels)
    condition = tf.less(residual, delta)
    small_res = 0.5 * tf.square(residual)
    large_res = delta * residual - 0.5 * tf.square(delta)
    return tf.select(condition, small_res, large_res)


## 4. Logistic Regression
### MNIST Database
![MNIST](figures/03_01.png)  

- Each image is a 28x28 array, flattened out to be a 1-d tensor of size 784
- X: image of a handwritten digit
- Y: the digit value
- goal: Recognize the digit in the image
- model: 
    - logits = X * w + b
    - Y_predicted = softmax(logits)
    - loss = cross_entropy(Y, Y_predicted)
    
### Batch ‘em up  
    X = tf.placeholder(tf.float32, [batch_size, 784], name="image")
    Y = tf.placeholder(tf.float32, [batch_size, 10], name="label")
### Process data
    from tensorflow.examples.tutorials.mnist import input_data
    
    # using TF Learn's built in function to load MNIST data to the folder data/mnist
    MNIST = input_data.read_data_sets("/data/mnist", one_hot=True)
    
    # MNIST.train: 55,000 examples
    # MNIST.validation: 5,000 examples
    # MNIST.test: 10,000 examples
    
### Phase 1: Assemble our graph
- Step 2: Create placeholders for inputs and labels
        X = tf.placeholder(tf.float32, [batch_size, 784], name="image")
        Y = tf.placeholder(tf.float32, [batch_size, 10], name="label")
- Step 3: Create weight and bias
        w = tf.Variable(tf.random_normal(shape=[784, 10], stddev=0.01), name='weights')
        b = tf.Variable(tf.zeros([1, 10]), name="bias")
- Step 4: Build model to predict Y  
        # the model that returns the logits.
            # this logits will be later passed through softmax layer
        logits = tf.matmul(X, w) + b 
- Step 5: Specify loss function 
        # use cross entropy of softmax of logits as the loss function
        entropy = tf.nn.softmax_cross_entropy_with_logits(logits, Y, name='loss')
        loss = tf.reduce_mean(entropy) # computes the mean over all the examples in the batch
- Step 6: Create optimizer
        optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
        
    - add new operations to the computation graph: ones to compute gradients, compute parameter update steps, and apply update steps to the parameters
    - when run, will apply the gradient descent updates to the parameters. Training the model can therefore be accomplished by repeatedly running optimizer



### Phase 2: Train our model

(https://www.tensorflow.org/get_started/mnist/pros)  

        with tf.Session() as sess:
            # to visualize using TensorBoard
            writer = tf.summary.FileWriter('./my_graph/03/logistic_reg', sess.graph)

            start_time = time.time()
            sess.run(tf.global_variables_initializer())	
            n_batches = int(mnist.train.num_examples/batch_size)
            for i in range(n_epochs): # train the model n_epochs times
                total_loss = 0

                for _ in range(n_batches):
                    X_batch, Y_batch = mnist.train.next_batch(batch_size)
                    _, loss_batch = sess.run([optimizer, loss], feed_dict={X: X_batch, Y:Y_batch}) 
                    total_loss += loss_batch
                print 'Average loss epoch {0}: {1}'.format(i, total_loss/n_batches)

            print 'Total time: {0} seconds'.format(time.time() - start_time)

            print('Optimization Finished!') # should be around 0.35 after 25 epochs

            # test the model
            n_batches = int(mnist.test.num_examples/batch_size)
            total_correct_preds = 0
            for i in range(n_batches):
                X_batch, Y_batch = mnist.test.next_batch(batch_size)
                _, loss_batch, logits_batch = sess.run([optimizer, loss, logits], feed_dict={X: X_batch, Y:Y_batch}) 
                preds = tf.nn.softmax(logits_batch)
                correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(Y_batch, 1))
                accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32)) # need numpy.count_nonzero(boolarr) :(
                total_correct_preds += sess.run(accuracy)	

            print 'Accuracy {0}'.format(total_correct_preds/mnist.test.num_examples)

            writer.close()